In [1]:
import torch
from config import BitformerConfig
from model_zoo import VisionBitformerForImageClassification, VisionBitformerForSemanticSegmentation
from metrics import compute_metrics_single_label_classification
from trainer import get_trainer
from data_zoo import get_vision_dataset, vision_collator
from utils import get_yaml

In [2]:
yaml_path = './yamls/vision/small.yaml'
args = get_yaml(yaml_path)

In [3]:
cfg = BitformerConfig(**args['model_config'], num_labels=args['general_config']['num_labels'])
model = VisionBitformerForSemanticSegmentation(config=cfg)
model.num_parameters() / 1e6

1.246164

In [4]:
model

VisionBitformerForSemanticSegmentation(
  (model): VisionBitformer(
    (embed_tokens): Embedding(32000, 28)
    (layers): ModuleList(
      (0): BitformerLayer(
        (self_attn): VisionAttention(
          (qkv): BitLinear(
            in_features=84, out_features=2304, bias=False
            (norm): LayerNorm((84,), eps=1e-05, elementwise_affine=True)
          )
          (o): BitLinear(
            in_features=768, out_features=84, bias=False
            (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
        )
        (MLP): VisionMoeBlock(
          (router): Linear(in_features=84, out_features=4, bias=False)
          (experts): ModuleList(
            (0-3): 4 x MLP(
              (w1): BitLinear(
                in_features=28, out_features=256, bias=False
                (norm): LayerNorm((28,), eps=1e-05, elementwise_affine=True)
              )
              (w2): BitLinear(
                in_features=256, out_features=28, bias=False
          

In [5]:
test = torch.rand(8, 3, 28, 28)
labels = torch.randint(0, 9, (8, 28, 28))
out = model(test, labels=labels)

torch.Size([8, 3, 28, 28])
torch.Size([8, 10, 28, 28])


In [8]:
out.logits.shape

torch.Size([8, 10, 28, 28])

In [10]:
data_path = 'mnist'
train_dataset, test_dataset = get_vision_dataset(data_path)

In [11]:
trainer = get_trainer(model=model,
                      train_dataset=train_dataset,
                      valid_dataset=test_dataset,
                      compute_metrics=compute_metrics_single_label_classification,
                      data_collator=vision_collator,
                      **args['training_args'])

In [12]:
trainer.train()
trainer.evaluate()

  0%|          | 0/9380 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 1.0212, 'grad_norm': 1.5853161811828613, 'learning_rate': 0.0004900072000534774, 'epoch': 1.0}


  0%|          | 0/157 [00:00<?, ?it/s]

Checkpoint destination directory ./results\checkpoint-938 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.4045952260494232, 'eval_f1': 0.8887720066014907, 'eval_precision': 0.8905850300103851, 'eval_recall': 0.8885, 'eval_accuracy': 0.8885, 'eval_runtime': 0.939, 'eval_samples_per_second': 10649.391, 'eval_steps_per_second': 167.195, 'epoch': 1.0}


KeyboardInterrupt: 